In [ ]:
import os
from faker import Faker
import random
from django.db import IntegrityError
print(os.environ.get('DJANGO_SETTINGS_MODULE'))
print(os.environ.get("DJANGO_ALLOW_ASYNC_UNSAFE"))

from django.contrib.auth.models import Group
# from profile.models import User

fake = Faker()
# generate users by Faker


def user_faker():
    u = {}
    u['username'] = fake.unique.user_name()
    u['password'] = fake.password()
    u['is_superuser'] = False
    u['is_staff'] = False
    return u


def user_generator(profile):
    # user = User.objects.create_user(**u)
    user = User(**profile)
    return user

def group_generator(group_name):
    try:
        group = Group.objects.create(name=group_name)
    except IntegrityError:
        print(f"group {group_name} is exist!")
        group = Group.objects.get(name=group_name)
    return group


# if save=True, the uses will save to db
def add_bulk_user(user_num, group_name="nogroup", save=False):
    print(f"****** start add {user_num} users ******")
    for _ in range(user_num):
        profile = user_faker()
        user = user_generator(profile)
        user.set_password(profile['password'])
        if save:
            user.save()
            if group_name !="nogroup":
                group = group_generator(group_name=group_name)
                user.groups.add(group)
            print(f"save {user.username} to group {group_name}")
        else:
            print(f"test only | add {user.username} to group {group_name}")
            
    if not save:
        print("****** Test Task End, Please use save=True params to save to db ******\n\n")
    else:
        print("****** End ******\n\n")
        

# insert first 3 users to db
def add_default_user():
    
    admin = {'username': 'admin', 'password': '123456',
             'is_superuser': True, 'is_staff': True}
    foo = {'username': 'foo', 'password': '123456',
           'is_superuser': False, 'is_staff': True}
    bar = {'username': 'bar', 'password': '123456',
           'is_superuser': False, 'is_staff': False}
    cus = {'username': 'cus', 'password': '123456',
           'is_superuser': False, 'is_staff': False}
    group_list= ['manager', 'staff', 'user']
    
    for profile, group_name in zip([admin, foo, bar], group_list):
        user = user_generator(profile)
        
        try:
            user.set_password('123456')
            user.save()
        except IntegrityError:
            print(f"user {profile['username']} is exist")
            user = User.objects.get(username=profile['username'])
        
        group = group_generator(group_name)
        user.groups.add(group)

    
    print("****** End ******\n\n")


In [ ]:
add_default_user()

In [ ]:
group = Group.objects.get(name="manager")
admin = User.objects.get(username='admin')

In [ ]:
admin.groups.all()

In [ ]:
# add_bulk_user(user_num=10, group_name='user', save=True)
add_bulk_user(user_num=10, group_name='user')   #test